In [1]:

!pip install pycuda

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 22.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 11.0 MB/s eta 0:00:00
  Created wheel for pycuda: filename=pycuda-2025.1-cp311-cp311-linux_x86_64.whl size=660427 sha256=8fd70b8f5dd9f8dd5602bbc1f9dfab6f5a3b4a37e63830f839ebf6a88ae4b705
  Stored in directory: /root/.cache/pip/wheels/77/7e/6c/d2d1451ea6424cdc3d67b36c16fa7111eafdf2034bc3405666
Successfully built pycuda


In [2]:
import pycuda.driver as cuda
import pycuda.autoinit
import numpy as np
from pycuda.compiler import SourceModule

# Step 1: Get dynamic input for matrix size
N = int(input("Enter the matrix size N: "))  # Dynamic input for matrix size

# Step 2: Get dynamic input for matrices A and B
A = np.array([list(map(float, input(f"Enter row {i+1} of matrix A (space-separated): ").split())) for i in range(N)], dtype=np.float32)
B = np.array([list(map(float, input(f"Enter row {i+1} of matrix B (space-separated): ").split())) for i in range(N)], dtype=np.float32)

# Initialize the result matrix C with zeros
C = np.zeros((N, N), dtype=np.float32)

# Flatten matrices for the device (GPU)
A = A.flatten()
B = B.flatten()
C = C.flatten()

# Step 3: Allocate memory on the device for matrices A, B, and C
A_gpu = cuda.mem_alloc(A.nbytes)
B_gpu = cuda.mem_alloc(B.nbytes)
C_gpu = cuda.mem_alloc(C.nbytes)

# Step 4: Copy matrices from host (CPU) to device (GPU)
cuda.memcpy_htod(A_gpu, A)
cuda.memcpy_htod(B_gpu, B)

# CUDA C kernel code for matrix multiplication (C = A * B)
kernel_code = """
__global__ void matmul(float *A, float *B, float *C, int N)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N)
    {
        float sum = 0.0;
        for (int k = 0; k < N; ++k)
        {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}
"""

# Step 5: Compile the CUDA code and get the kernel function
mod = SourceModule(kernel_code)
matmul = mod.get_function("matmul")

# Step 6: Set up block and grid size for the kernel
block_size = 2  # You can adjust this value for better performance
grid_size = int(np.ceil(N / block_size))

# Step 7: Launch the CUDA kernel
matmul(A_gpu, B_gpu, C_gpu, np.int32(N), block=(block_size, block_size, 1), grid=(grid_size, grid_size))

# Step 8: Copy the result matrix C from device (GPU) to host (CPU)
cuda.memcpy_dtoh(C, C_gpu)

# Step 9: Reshape and print the result matrix C
C = C.reshape(N, N)
print("Result Matrix C:")
print(C)


Enter the matrix size N: 3
Enter row 1 of matrix A (space-separated): 2 2 2
Enter row 2 of matrix A (space-separated): 2 2 2
Enter row 3 of matrix A (space-separated): 2 2 2
Enter row 1 of matrix B (space-separated): 3 3 3
Enter row 2 of matrix B (space-separated): 3 3 3
Enter row 3 of matrix B (space-separated): 3 3 3
Result Matrix C:
[[18. 18. 18.]
 [18. 18. 18.]
 [18. 18. 18.]]


CUDA (Compute Unified Device Architecture) is a parallel computing platform and programming model developed by NVIDIA. It allows developers to harness the power of NVIDIA GPUs to perform computations in parallel. CUDA enables parallel execution of code by breaking down tasks into smaller chunks that can be run concurrently on many threads across the GPU.

Key Concepts in CUDA Programming
Kernel: A kernel is a function written in CUDA C that runs on the GPU. It executes in parallel across many threads. In the context of matrix multiplication, the kernel performs the operation on each element of the resultant matrix.

Threads: A thread is the smallest unit of execution in CUDA. Each thread performs a computation, such as calculating one element of the result matrix. Threads are grouped into blocks to better manage execution.

Blocks: Threads are organized into blocks. Each block can contain a certain number of threads. The size of the block determines how many threads are launched at once. A block is the basic unit of work distribution in CUDA, and the kernel is executed across multiple blocks.

Grid: The grid is a collection of blocks. The size of the grid determines how many blocks are launched to cover the entire problem. It defines the total number of threads executing the kernel.

Memory Management: In CUDA, there are different types of memory:

Global memory: This is the main memory accessible by all threads in a grid.

Shared memory: Memory shared by all threads within a block. This is much faster than global memory but has limited size.

Device memory: Memory that resides on the GPU.

Key CUDA Code Snippets and Concepts
Matrix Multiplication Kernel

The first key part of the code is the CUDA kernel that performs matrix multiplication. Here is the kernel snippet:


__global__ void matmul(float *A, float *B, float *C, int N)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < N && col < N)
    {
        float sum = 0.0;
        for (int k = 0; k < N; ++k)
        {
            sum += A[row * N + k] * B[k * N + col];
        }
        C[row * N + col] = sum;
    }
}
Explanation:

The __global__ keyword designates the function as a kernel that can be executed on the GPU.

blockIdx and threadIdx are built-in variables that give the index of the block and thread within a block, respectively.

blockDim gives the dimensions of the block, i.e., how many threads are in each block.

C[row * N + col] calculates one element in the result matrix C based on the dot product of the row of matrix A and the column of matrix B.

Grid and Block Configuration

CUDA uses a grid of blocks to distribute the workload across multiple threads. The number of blocks and threads per block are defined based on the matrix size. Here’s the configuration in the code:

block_size = 2  # Number of threads per block in each dimension
grid_size = int(np.ceil(N / block_size))  # Number of blocks in each dimension
Explanation:

The matrix is divided into smaller blocks, each of size block_size. For example, block_size = 2 means that each block will have 2 threads in both the x and y directions.

The grid size (grid_size) is computed to ensure that the total number of blocks can cover all elements of the matrix.

Memory Allocation and Data Transfer

Before calling the kernel, the matrices are transferred from the host (CPU) to the device (GPU) memory. Here’s how the memory allocation and data transfer work:


A_gpu = cuda.mem_alloc(A.nbytes)
B_gpu = cuda.mem_alloc(B.nbytes)
C_gpu = cuda.mem_alloc(C.nbytes)

cuda.memcpy_htod(A_gpu, A)
cuda.memcpy_htod(B_gpu, B)
Explanation:

cuda.mem_alloc() allocates memory on the GPU to store the matrices.

cuda.memcpy_htod() transfers the data from the host (CPU) memory to the device (GPU) memory.

The matrices A, B, and C are transferred to the GPU so the kernel can perform the computation.

Kernel Launch

The kernel is launched with the following code:


matmul(A_gpu, B_gpu, C_gpu, np.int32(N), block=(block_size, block_size, 1), grid=(grid_size, grid_size))
Explanation:

The matmul kernel is called with the matrices A_gpu, B_gpu, and C_gpu in GPU memory.

The block size and grid size are specified as (block_size, block_size, 1) for 2D threading in both x and y directions. The grid size is determined based on the matrix size N.

The kernel execution begins once this line is reached, and each thread calculates one element of the result matrix C.

Copying Data Back to Host

After the kernel completes, the result is copied back to the host (CPU) memory for further processing or printing:


cuda.memcpy_dtoh(C, C_gpu)
Explanation:

cuda.memcpy_dtoh() transfers the data from the device (GPU) back to the host (CPU).

After the result is copied, it is reshaped into a 2D matrix and printed.

Summary of CUDA Concepts
Parallel Execution: The key benefit of CUDA is its ability to execute thousands of threads in parallel, each computing a small portion of the matrix multiplication task. This significantly speeds up operations compared to traditional CPU-based computations.

Thread and Block Structure: By organizing threads into blocks, CUDA efficiently manages parallel execution. Each thread computes one element of the result matrix, while the blocks are processed concurrently.

Memory Management: Efficient memory management is crucial for CUDA programs. In this code, memory is allocated on the GPU, and data is transferred between host and device memory as needed.

Optimization: The choice of block size and grid size can impact performance. A block size of 2x2 threads is used in this example, but larger blocks might be more efficient depending on the GPU architecture.